In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [ ]:
# Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
# def get_list_of_university_towns():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
#     return "ANSWER"
# ut=pd.read_fwf('university_towns.txt')

In [ ]:
#Now to put it in one function
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list.''' 
    ut=pd.read_csv('university_towns.txt', sep='|', header=None, names=['RegionName'])
    ut['State']=ut[ut['RegionName'].str.match('.*\[edit\]$')]
    ut['State'].fillna(method='ffill', inplace=True)
    ut = ut.reindex(columns=['State', 'RegionName']) #instead of.set_index('State').reset_index()
    ut = ut[ut['RegionName'].str.match('(?!.*\[edit\]$)')]
    ut['State']=ut['State'].str.extract('(.*)(?:\[edit\])$')
    ut['RegionName']=ut['RegionName'].str.replace('\d+| \(.*','')

    for acronym, name in states.items():
        ut['State'].loc[ut['State'] == name] = acronym

    
    return ut
get_list_of_university_towns()

In [ ]:
# def get_recession_start():
#     '''Returns the year and quarter of the recession start time as a 
#     string value in a format such as 2005q3'''
    
#     return "ANSWER"

In [ ]:
df=pd.read_excel('gdplev.xls', skiprows=list(range(5))+[6,7], usecols=list(range(3))+list(range(4,7)))
#i keep it here for future reference but we have two distinct dataframes.
df=df.rename(columns={'Unnamed: 0':'Year','Unnamed: 4':'Quarter'})
quarters=df.loc[:,'Quarter':'GDP in billions of chained 2009 dollars.1']
quarters2000=quarters[(quarters['Quarter'].str.extract('(\d+)',expand=False).astype('float')>=2000)]
quarters2000.reset_index(inplace=True, drop=True)
quarters2000

In [ ]:
# quarters2000[(quarters2000['GDP in billions of current dollars.1'].shift(1)-quarters['GDP in billions of current dollars.1'])>0]
indexplus2=quarters2000[((quarters2000['GDP in billions of current dollars.1'].shift(2)-quarters2000['GDP in billions of current dollars.1'].shift(1))>0) & ((quarters2000['GDP in billions of current dollars.1'].shift(1)-quarters2000['GDP in billions of current dollars.1'])>0 )].index[0]
start_crisis=quarters2000.iloc[indexplus2-2,quarters2000.columns.get_loc('Quarter')]
start_crisis


In [ ]:
def clean_gdp():
    df=pd.read_excel('gdplev.xls', skiprows=list(range(5))+[6,7], usecols=list(range(3))+list(range(4,7)))
    #i keep it here for future reference but we have two distinct dataframes.
    df=df.rename(columns={'Unnamed: 0':'Year','Unnamed: 4':'Quarter'})
    quarters=df.loc[:,'Quarter':'GDP in billions of chained 2009 dollars.1']
    quarters2000=quarters[(quarters['Quarter'].str.extract('(\d+)',expand=False).astype('float')>=2000)]
    quarters2000.reset_index(inplace=True, drop=True)
    return quarters2000
clean_gdp()

In [ ]:
#so for the function
def get_recession_start():
    quarters2000=clean_gdp()
    indexplus2=quarters2000[((quarters2000['GDP in billions of current dollars.1'].shift(2)-quarters2000['GDP in billions of current dollars.1'].shift(1))>0) & ((quarters2000['GDP in billions of current dollars.1'].shift(1)-quarters2000['GDP in billions of current dollars.1'])>0 )].index[0]
    start_crisis=quarters2000.iloc[indexplus2-2,quarters2000.columns.get_loc('Quarter')]
    return start_crisis
get_recession_start()

In [ ]:
def get_recession_end():
    quarters2000=clean_gdp()
    rec_start=get_recession_start()
    ind_rec_start=quarters2000[quarters2000['Quarter']== rec_start].index[0]
    indexout=quarters2000[((quarters2000['GDP in billions of current dollars.1'].shift(2)-quarters2000['GDP in billions of current dollars.1'].shift(1))<0) & ((quarters2000['GDP in billions of current dollars.1'].shift(1)-quarters2000['GDP in billions of current dollars.1'])<0) & (quarters2000.index>ind_rec_start)].index[0]
    stopcrisis = quarters2000.iloc[indexout,quarters2000.columns.get_loc('Quarter')]
    return stopcrisis

get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quarters2000=clean_gdp()
    rec_start=get_recession_start() 
    rec_end= get_recession_end()
    ind_rec_start=quarters2000[quarters2000['Quarter']== rec_start].index[0]
    ind_rec_end=quarters2000[quarters2000['Quarter']== rec_end].index[0] 
    ind_bottom = quarters2000['GDP in billions of current dollars.1'].iloc[ind_rec_start:ind_rec_end+1].idxmin()
    bottomcrisis = quarters2000.iloc[ind_bottom,quarters2000.columns.get_loc('Quarter')]
    return bottomcrisis
get_recession_bottom()

In [ ]:
# def convert_housing_data_to_quarters():
#     '''Converts the housing data to quarters and returns it as mean 
#     values in a dataframe. This dataframe should be a dataframe with
#     columns for 2000q1 through 2016q3, and should have a multi-index
#     in the shape of ["State","RegionName"].
    
#     Note: Quarters are defined in the assignment description, they are
#     not arbitrary three month periods.
    
#     The resulting dataframe should have 67 columns, and 10,730 rows.
#     '''
    
#     return "ANSWER"

In [ ]:
df=pd.read_csv('City_Zhvi_AllHomes.csv', encoding = "ISO-8859-1")
df

In [ ]:
groupby_dict={}
for i in range(0,17):
    for j in range(1,13):
        if(j<=3):
            groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q1')
        elif(j<=6):
            groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q2')
        elif(j<=9):
            groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q3')
        else:
            groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q4')

groupby_dict

In [ ]:
# dd=df.set_index('RegionID').groupby(groupby_dict,axis=1)

df.set_index(['State','RegionName']).loc[:,'2000-01':].groupby(groupby_dict,axis=1).mean()


In [ ]:
def convert_housing_data_to_quarters():
    df=pd.read_csv('City_Zhvi_AllHomes.csv', encoding = "ISO-8859-1")
    groupby_dict={}
    for i in range(0,17):
        for j in range(1,13):
            if(j<=3):
                groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q1')
            elif(j<=6):
                groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q2')
            elif(j<=9):
                groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q3')
            else:
                groupby_dict[(str(2000+i)+ '-'+'{:0>2d}'.format(j))]=(str(2000+i)+ 'q4')

    
    return df.set_index(['State','RegionName']).loc[:,'2000-01':].groupby(groupby_dict,axis=1).mean()
convert_housing_data_to_quarters()    

In [ ]:
# def run_ttest():
#     '''First creates new data showing the decline or growth of housing prices
#     between the recession start and the recession bottom. Then runs a ttest
#     comparing the university town values to the non-university towns values, 
#     return whether the alternative hypothesis (that the two groups are the same)
#     is true or not as well as the p-value of the confidence. 
    
#     Return the tuple (different, p, better) where different=True if the t-test is
#     True at a p<0.01 (we reject the null hypothesis), or different=False if 
#     otherwise (we cannot reject the null hypothesis). The variable p should
#     be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#     value for better should be either "university town" or "non-university town"
#     depending on which has a lower mean price ratio (which is equivilent to a
#     reduced market loss).'''
    
#     return "ANSWER"

In [ ]:
df=convert_housing_data_to_quarters()
df=df.loc[:,get_recession_start():get_recession_end()]
df

In [ ]:
ut=get_list_of_university_towns().set_index(['State', 'RegionName'])
ut['University Town']=True
ut


In [ ]:
comb = df.merge(ut,how='left',right_index=True,left_index=True)
comb['University Town']=comb['University Town'].fillna(False)
comb

In [ ]:
university=comb[comb['University Town']==True]
nonuniversity=comb[comb['University Town']==False]
university_decline=university.iloc[:,-2]-university.iloc[:,0]
nonuniversity_decline=nonuniversity.iloc[:,-2]-nonuniversity.iloc[:,0]
print(university_decline.mean()/university.iloc[:,0].mean())
print(nonuniversity_decline.mean()/nonuniversity.iloc[:,0].mean())

In [ ]:
ttest_ind(university_decline, nonuniversity_decline,nan_policy='omit')

In [ ]:
def run_ttest():
    ut=get_list_of_university_towns().set_index(['State', 'RegionName'])
    ut['University Town']=True

    df=convert_housing_data_to_quarters()
    df=df.loc[:,get_recession_start():get_recession_end()]

    comb = df.merge(ut,how='left',right_index=True,left_index=True)
    comb['University Town']=comb['University Town'].fillna(False)

    university=comb[comb['University Town']==True]
    nonuniversity=comb[comb['University Town']==False]
    university_decline=university.iloc[:,-2]-university.iloc[:,0]
    nonuniversity_decline=nonuniversity.iloc[:,-2]-nonuniversity.iloc[:,0]
    univ_dec_ratio=university_decline.mean()/university.iloc[:,0].mean()
    nonuniv_dec_ratio=nonuniversity_decline.mean()/nonuniversity.iloc[:,0].mean()
    better = ('university-town' if univ_dec_ratio>nonuniv_dec_ratio else 'non-university-town')
    (stat,p)=ttest_ind(university_decline, nonuniversity_decline,nan_policy='omit')
    different = (True if p<0.01 else False)

    return(different,p,better)

run_ttest()
